# Bias Analysis of Sentiment Analysis Models and Datasets

The process of bias analysis is done in 3 steps:


1.   First, train and test on a logistic regression model
2.   Second, fine tune and test a standard dataset on another model
3.   Lastly, train the same model from step 2 on a toxicity dataset

Following these 3 steps, we then analyse the bias either inherent in the model or gradually learnt from the training in the provided datasets.



# 1. Basic test of Bias using Logistic Regression

We first create a baseline model in which we test whether any kind of bias exists in a simple model such as a logistic regression model. This model is trained on the Stanford Sentiment Treebank v2 (SST2) dataset and then tested on the Equity Evaluation Corpus (EEC) dataset.

This is then further utilized as a basis for bias analysis in our control model and then our actual testing model

### For obtaining the datasets from kaggle:

In [28]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("jkhanbk1/sst2-dataset")

print("Path to dataset files:", path)

100%|██████████| 827k/827k [00:00<00:00, 114MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/jkhanbk1/sst2-dataset/versions/1


In [32]:
import os
# for the csv datasets
print(os.listdir('/root/.cache/kagglehub/datasets/jkhanbk1/sst2-dataset/versions/1/Finalv SST-2 dataset CSV format'))


['train.csv', 'unsup.csv', 'test.csv', 'val.csv']


In [35]:
import pandas as pd

train_data = '/root/.cache/kagglehub/datasets/jkhanbk1/sst2-dataset/versions/1/Finalv SST-2 dataset CSV format/train.csv'
test_data = '/root/.cache/kagglehub/datasets/jkhanbk1/sst2-dataset/versions/1/Finalv SST-2 dataset CSV format/test.csv'
val_data = '/root/.cache/kagglehub/datasets/jkhanbk1/sst2-dataset/versions/1/Finalv SST-2 dataset CSV format/val.csv'

train_df = pd.read_csv(train_data)
test_df = pd.read_csv(test_data)
val_df = pd.read_csv(val_data)

print(train_df.columns)

Index(['label', 'sentence'], dtype='object')


### Now the actual definition of the Logistic regression model along with its loss function and optimizer are as follows:

In [56]:
import re, math, hashlib
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# Configure device to use gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyper-parameters
input_dim = 20000
learning_rate = 1e-3
num_epochs = 25


# Simpler tokenizer and hashing vectorizer
token_pat = re.compile(r"\w+")
def tokenize(text):
  return token_pat.findall(str(text).lower())

class HashingVectorizer:
  def __init__(self, n_features=20000):
    self.n_features = n_features

  def _idx(self, token):
    return int(hashlib.md5(token.encode("utf-8")).hexdigest(), 16) % self.n_features

  def transform_one(self, text):
    x = torch.zeros(self.n_features, dtype=torch.float32)
    for tok in tokenize(text):
        x[self._idx(tok)] += 1.0
    n = torch.linalg.norm(x)
    if n > 0: x /= n
    return x

vectorizer = HashingVectorizer(input_dim)


# To wrap the pandas dataframe so that torch can read from it better
class FrameDataset(Dataset):
  def __init__(self, df, text_col='sentence', label_col='label'):
    self.texts = df[text_col].tolist()
    self.labels = df[label_col].tolist()

  def __len__(self):
    return len(self.texts)

  def __getitem__(self, i):
    return vectorizer.transform_one(self.texts[i]), torch.tensor([self.labels[i]], dtype=torch.float32)

train_ds = FrameDataset(train_df)
test_ds   = FrameDataset(test_df)
train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)  # randomizes each epoch
test_loader   = DataLoader(test_ds,   batch_size=256, shuffle=False) # deterministic epochs


class LogisticRegression(nn.Module):
  def __init__(self, input_size):
    super(LogisticRegression, self).__init__()
    self.linear = nn.Linear(input_size, 1)

  def forward(self, x):
    y_predicted = torch.sigmoid(self.linear(x))
    return y_predicted


model = LogisticRegression(input_dim).to(device)
criterion = nn.BCELoss()  # Loss Function
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


### Model Training Loop

In [57]:
for epoch in range(num_epochs):
  # Train the model
  model.train()
  total_loss = 0.0
  for xb, yb in train_loader:
    xb, yb = xb.to(device), yb.to(device)
    optimizer.zero_grad()
    out_logits = model(xb)
    loss = criterion(out_logits, yb)
    loss.backward()
    optimizer.step()
    total_loss += loss.item() * xb.size(0)
  avg_loss = total_loss / len(train_loader.dataset)

# Evaluate the model in the current epoch
  model.eval()
  correct, total = 0, 0
  with torch.no_grad():
    for xb, yb in test_loader:
      xb, yb = xb.to(device), yb.to(device)
      preds = (torch.sigmoid(model(xb)) >= 0.5).long()
      correct += (preds == yb.long()).sum().item()
      total += yb.size(0) # no of elements in the tensor
    acc = correct / total
    print(f"Epoch {epoch}: train_loss={avg_loss:.4f}  val_acc={acc:.4f}")





Epoch 0: train_loss=0.6901  val_acc=0.4992
Epoch 1: train_loss=0.6820  val_acc=0.4992
Epoch 2: train_loss=0.6745  val_acc=0.4992
Epoch 3: train_loss=0.6673  val_acc=0.4992
Epoch 4: train_loss=0.6605  val_acc=0.4992
Epoch 5: train_loss=0.6539  val_acc=0.4992
Epoch 6: train_loss=0.6474  val_acc=0.4992
Epoch 7: train_loss=0.6411  val_acc=0.4992
Epoch 8: train_loss=0.6349  val_acc=0.4992
Epoch 9: train_loss=0.6290  val_acc=0.4992
Epoch 10: train_loss=0.6232  val_acc=0.4992
Epoch 11: train_loss=0.6175  val_acc=0.4992
Epoch 12: train_loss=0.6119  val_acc=0.4992
Epoch 13: train_loss=0.6065  val_acc=0.4992
Epoch 14: train_loss=0.6012  val_acc=0.4992
Epoch 15: train_loss=0.5960  val_acc=0.4992
Epoch 16: train_loss=0.5910  val_acc=0.4992
Epoch 17: train_loss=0.5860  val_acc=0.4992
Epoch 18: train_loss=0.5812  val_acc=0.4992
Epoch 19: train_loss=0.5765  val_acc=0.4992
Epoch 20: train_loss=0.5719  val_acc=0.4992
Epoch 21: train_loss=0.5673  val_acc=0.4992
Epoch 22: train_loss=0.5629  val_acc=0.499

In [ ]:
def predict(sentence: str):
    x = vectorizer.transform_one(sentence).unsqueeze(0).to(device)
    with torch.no_grad():
        p = torch.sigmoid(model(x)).item()
    return p, ("positive" if p >= 0.5 else "negative")
